In [17]:
import os
import sys

if os.getcwd().split('/')[-1] == 'notebooks':
    os.chdir('../')
os.getcwd()

'/home/claude/development/QueryOptDeepDive'

In [18]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import string
from mo_sql_parsing import parse

In [19]:
base_dirs = dict()

for root, dirs, files in os.walk('data/stack/so_queries'):
    for file in files:
        file_path = os.path.join(root, file)
        
        split_path = file_path.split('/')
        base_query = split_path[-2]

        if base_query not in base_dirs.keys():
            base_dirs[base_query] = list()

        base_dirs[base_query].append(file_path)
        

In [24]:
queries = {}

for file in os.listdir('bao/queries/stack'):
    base_query, query_name = file.split('__')
    #print(base_query, '\t', query_name)

    if base_query in ['q9', 'q10']:
        continue

    if base_query not in queries.keys():
        queries[base_query] = list()

    queries[base_query].append((base_query, query_name))

all_queries = []
for base_query in sorted(queries.keys()):
    print('__'.join(queries[base_query][0]))

    for x in queries[base_query]:
        all_queries.append(x)

print(f"# Queries: {len(all_queries)}")

q1__q1-067.sql
q11__c1ae2a992cde4ea2c4922d852df22043254b4f84.sql
q12__55de941e8497cfeeb93d3f8f2d7a18489e0e6c32.sql
q13__a3d03772d880754fc4e150d82908757477ae2186.sql
q14__5dbc1d1f1a0467ad0086e6cb337144387a37533a.sql
q15__3e37e62655ceaebc14e79edad518e5710752f51d.sql
q16__ea9efde510227beb8d624b8c4a6941b9d5e6e637.sql
q2__q2-035.sql
q3__q3-068.sql
q4__q4-002.sql
q5__q5-052.sql
q6__q6-067.sql
q7__q7-034.sql
q8__q8-096.sql
# Queries: 112


In [25]:
for q_dir, q_file in all_queries:
    file_path = f'data/stack/so_queries/{q_dir}/{q_file.replace(".sql.sql",".sql")}'
    with open(file_path, 'r') as f:
        lines = f.readlines()
    sql = ''.join(lines)

    query_ident = '__'.join([file_path.split('/')[-2], file_path.split('/')[-1]])

    os.makedirs('bao/queries/stack', exist_ok=True)
    file_path = f"bao/queries/stack/{query_ident}"
    with open(file_path, 'w') as f:
        f.write(sql)
    
    os.makedirs('balsa/queries/stack', exist_ok=True)
    file_path = f"balsa/queries/stack/{query_ident}"
    with open(file_path, 'w') as f:
        f.write(sql)
    

### Random Sampling

In [27]:
tmp = []

for q_dir, q_ident in all_queries:
    file_path = f'data/stack/so_queries/{q_dir}/{q_ident.replace(".sql.sql",".sql")}'
    with open(file_path, 'r') as f:
        lines = f.readlines()
    sql = ''.join(lines)
    
    tmp.append({
        'query_ident': q_ident,
        'query_template': q_dir,
        'full_query_file_name': f"{q_dir}__{q_ident}",
        'sql': sql,
        'file_path': file_path
    })

df = pd.DataFrame(tmp).sort_values(['query_template', 'query_ident']).reset_index(drop=True)
df

,query_ident,query_template,full_query_file_name,sql,file_path
0,q1-009.sql,q1,q1__q1-009.sql,"\nselect count(*) from tag, site, question, ta...",data/stack/so_queries/q1/q1-009.sql
1,q1-031.sql,q1,q1__q1-031.sql,"\nselect count(*) from tag, site, question, ta...",data/stack/so_queries/q1/q1-031.sql
2,q1-035.sql,q1,q1__q1-035.sql,"\nselect count(*) from tag, site, question, ta...",data/stack/so_queries/q1/q1-035.sql
3,q1-067.sql,q1,q1__q1-067.sql,"\nselect count(*) from tag, site, question, ta...",data/stack/so_queries/q1/q1-067.sql
4,q1-075.sql,q1,q1__q1-075.sql,"\nselect count(*) from tag, site, question, ta...",data/stack/so_queries/q1/q1-075.sql
...,...,...,...,...,...
107,q8-062.sql,q8,q8__q8-062.sql,"\nselect count(distinct q1.id) from\nsite, pos...",data/stack/so_queries/q8/q8-062.sql
108,q8-065.sql,q8,q8__q8-065.sql,"\nselect count(distinct q1.id) from\nsite, pos...",data/stack/so_queries/q8/q8-065.sql
109,q8-074.sql,q8,q8__q8-074.sql,"\nselect count(distinct q1.id) from\nsite, pos...",data/stack/so_queries/q8/q8-074.sql
110,q8-076.sql,q8,q8__q8-076.sql,"\nselect count(distinct q1.id) from\nsite, pos...",data/stack/so_queries/q8/q8-076.sql


In [32]:
def save_queries(df: pd.DataFrame, base_target_dir: str, test: bool = False):
    target_dir = os.path.join(base_target_dir, ('test' if test else 'train'))
    os.makedirs(target_dir, exist_ok=True)

    for i, row in df.iterrows():
        sql = row['sql']
        file_name = row['full_query_file_name']

        file_path = os.path.join(target_dir, file_name)
        with open(file_path, 'w') as f:
            f.writelines(sql)

    if test:
        print()
        print(target_dir)
        print(sorted(df['full_query_file_name'].tolist()))
    

In [33]:
n_test__random = int(df.shape[0] * 0.2) + 1

random_test_1 = df.sample(n=n_test__random, replace=False, random_state=10).sort_index()
random_train_1 = df[~df.index.isin(random_test_1.index)]
save_queries(random_train_1, 'bao/queries/stack__random_split_1', test=False)
save_queries(random_test_1, 'bao/queries/stack__random_split_1', test=True)

"""
random_test_2 = df.sample(n=n_test__random, replace=False, random_state=20).sort_index()
random_train_2 = df[~df.index.isin(random_test_2.index)]
save_queries(random_train_2, 'bao/queries/stack__random_split_2', test=False)
save_queries(random_test_2, 'bao/queries/stack__random_split_2', test=True)

random_test_3 = df.sample(n=n_test__random, replace=False, random_state=30).sort_index()
random_train_3 = df[~df.index.isin(random_test_3.index)]
save_queries(random_train_3, 'bao/queries/stack__random_split_3', test=False)
save_queries(random_test_3, 'bao/queries/stack__random_split_3', test=True)
"""


bao/queries/stack__random_split_1/test
['q11__6c5cba419c5b7b02d431aeb5e766d775d812967a.sql', 'q11__c1ae2a992cde4ea2c4922d852df22043254b4f84.sql', 'q12__55de941e8497cfeeb93d3f8f2d7a18489e0e6c32.sql', 'q14__63c0776f1727638316b966fe748df7cc585a335b.sql', 'q14__74fd1af68d23f0690e3d0fc80bd9b42fa90a7e94.sql', 'q14__97e68ad5c2ced4c182366b3118a1f5f69b423fa6.sql', 'q14__b49361f85785200ed6ec1f2eec357b7598c9e564.sql', 'q15__3e37e62655ceaebc14e79edad518e5710752f51d.sql', 'q15__543ab3f730e494a69e3d15e59675f491544cb15d.sql', 'q15__b8ddf65b0c0c7867a9b560e571d457fec410715c.sql', 'q15__d5546c01928a687eb1f54e9f8eb4e1aff68fc381.sql', 'q16__1e863562a79ca1f7754c759ebab6a2addda0bde8.sql', 'q16__ea9efde510227beb8d624b8c4a6941b9d5e6e637.sql', 'q16__ed2ffeaefcf5ad8bbadc713ccc766541e12080aa.sql', 'q1__q1-031.sql', 'q1__q1-035.sql', 'q4__q4-042.sql', 'q4__q4-064.sql', 'q4__q4-089.sql', 'q5__q5-032.sql', 'q6__q6-060.sql', 'q6__q6-064.sql', 'q7__q7-099.sql']


"\nrandom_test_2 = df.sample(n=n_test__random, replace=False, random_state=20).sort_index()\nrandom_train_2 = df[~df.index.isin(random_test_2.index)]\nsave_queries(random_train_2, 'bao/queries/stack__random_split_2', test=False)\nsave_queries(random_test_2, 'bao/queries/stack__random_split_2', test=True)\n\nrandom_test_3 = df.sample(n=n_test__random, replace=False, random_state=30).sort_index()\nrandom_train_3 = df[~df.index.isin(random_test_3.index)]\nsave_queries(random_train_3, 'bao/queries/stack__random_split_3', test=False)\nsave_queries(random_test_3, 'bao/queries/stack__random_split_3', test=True)\n"

### Leave One Out Sampling

In [34]:
def split_loo(df, random_state):
    test_idx = []
    for query_template, df_group in df.groupby(['query_template']):
        test_idx.append(df_group.sample(n=1, replace=False, random_state=random_state).iloc[0].name)

    return df[~df.index.isin(test_idx)], df[df.index.isin(test_idx)]

loo1_train, loo1_test = split_loo(df, random_state=10)
save_queries(loo1_train, 'bao/queries/stack__leave_one_out_split_1', test=False)
save_queries(loo1_test, 'bao/queries/stack__leave_one_out_split_1', test=True)

"""
loo2_train, loo2_test = split_loo(df, random_state=20)
save_queries(loo2_train, 'bao/queries/stack__leave_one_out_split_2', test=False)
save_queries(loo2_test, 'bao/queries/stack__leave_one_out_split_2', test=True)

loo3_train, loo3_test = split_loo(df, random_state=30)
save_queries(loo3_train, 'bao/queries/stack__leave_one_out_split_3', test=False)
save_queries(loo3_test, 'bao/queries/stack__leave_one_out_split_3', test=True)
"""


bao/queries/stack__leave_one_out_split_1/test
['q11__6c5cba419c5b7b02d431aeb5e766d775d812967a.sql', 'q12__547c6bf1994c9b2ba82a7ae32f4b051beabf46fd.sql', 'q13__935e2051bf80eeafe91aeb6eb719b6b64b9592c2.sql', 'q14__5e4835cd72aaa2d7be15b2a5ffa2e66156b3656f.sql', 'q15__543ab3f730e494a69e3d15e59675f491544cb15d.sql', 'q16__b1a96cd48ba297dd93bce73c27b491069ad7449f.sql', 'q1__q1-035.sql', 'q2__q2-032.sql', 'q3__q3-043.sql', 'q4__q4-041.sql', 'q5__q5-041.sql', 'q6__q6-060.sql', 'q7__q7-047.sql', 'q8__q8-046.sql']


"\nloo2_train, loo2_test = split_loo(df, random_state=20)\nsave_queries(loo2_train, 'bao/queries/stack__leave_one_out_split_2', test=False)\nsave_queries(loo2_test, 'bao/queries/stack__leave_one_out_split_2', test=True)\n\nloo3_train, loo3_test = split_loo(df, random_state=30)\nsave_queries(loo3_train, 'bao/queries/stack__leave_one_out_split_3', test=False)\nsave_queries(loo3_test, 'bao/queries/stack__leave_one_out_split_3', test=True)\n"

### Base Query Sampling

In [35]:
# 16 query templates @ STACK -> 13-3 ~ 80-20 split
def split_base_query(df, n_test_templates=3):
    test_templates = np.random.choice(df['query_template'].unique(), size=n_test_templates, replace=False)
    return df[~df['query_template'].isin(test_templates)], df[df['query_template'].isin(test_templates)]

np.random.seed(10)
base_query1_train, base_query1_test = split_base_query(df)
save_queries(base_query1_train, 'bao/queries/stack__base_query_split_1', test=False)
save_queries(base_query1_test, 'bao/queries/stack__base_query_split_1', test=True)

"""
np.random.seed(20)
base_query2_train, base_query2_test = split_base_query(df)
save_queries(base_query2_train, 'bao/queries/stack__base_query_split_2', test=False)
save_queries(base_query2_test, 'bao/queries/stack__base_query_split_2', test=True)

np.random.seed(30)
base_query3_train, base_query3_test = split_base_query(df)
save_queries(base_query3_train, 'bao/queries/stack__base_query_split_3', test=False)
save_queries(base_query3_test, 'bao/queries/stack__base_query_split_3', test=True)
"""


bao/queries/stack__base_query_split_1/test
['q13__13ad1b8c6bea4fda1892b9fa82cc1ceb9ceb85fc.sql', 'q13__1ddcc8650e17b292bc7344902baffc90c5ae5761.sql', 'q13__935e2051bf80eeafe91aeb6eb719b6b64b9592c2.sql', 'q13__a091adce62743b65c04532e98e8ff3d7e546ea77.sql', 'q13__a3d03772d880754fc4e150d82908757477ae2186.sql', 'q13__add0df9dccb2790c14508e19c9e0deb79fad6ea2.sql', 'q13__d383cd5b4aee7d3f73508e2a1fe5f6d0f7dd42a2.sql', 'q13__d4707be2adfdbc842f42acb1fc16e3a43faf7474.sql', 'q2__q2-001.sql', 'q2__q2-012.sql', 'q2__q2-032.sql', 'q2__q2-035.sql', 'q2__q2-050.sql', 'q2__q2-081.sql', 'q2__q2-094.sql', 'q2__q2-098.sql', 'q7__q7-034.sql', 'q7__q7-036.sql', 'q7__q7-047.sql', 'q7__q7-077.sql', 'q7__q7-082.sql', 'q7__q7-085.sql', 'q7__q7-095.sql', 'q7__q7-099.sql']


"\nnp.random.seed(20)\nbase_query2_train, base_query2_test = split_base_query(df)\nsave_queries(base_query2_train, 'bao/queries/stack__base_query_split_2', test=False)\nsave_queries(base_query2_test, 'bao/queries/stack__base_query_split_2', test=True)\n\nnp.random.seed(30)\nbase_query3_train, base_query3_test = split_base_query(df)\nsave_queries(base_query3_train, 'bao/queries/stack__base_query_split_3', test=False)\nsave_queries(base_query3_test, 'bao/queries/stack__base_query_split_3', test=True)\n"